In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

gpus

2025-05-28 17:07:14.502384: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 17:07:14.510424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748466434.519625    6626 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748466434.522383    6626 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748466434.529433    6626 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import evaluate

bleurt = evaluate.load("bleurt", config_name="bleurt-tiny-512")

/home/j/Documents/MLotsawa/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:tensorflow:Reading checkpoint /home/j/.cache/huggingface/metrics/bleurt/bleurt-tiny-512/downloads/extracted/8db54174b0fdf60211f160069a4be54b2c9c579766503c437be8f9534b74e7c1/bleurt-tiny-512.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


I0000 00:00:1748466436.966135    6626 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4156 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
INFO:tensorflow:BLEURT initialized.


In [3]:
from datasets import load_dataset

ds = load_dataset('billingsmoore/temp', split='train')

references = ds['en']
predictions = ds['base_predictions']

In [4]:
# === Compute BLEURT scores ===
results = bleurt.compute(predictions=predictions, references=references)

In [5]:
results

{'scores': [-0.1061796173453331,
  0.7649470567703247,
  0.7780003547668457,
  -0.10414030402898788,
  -0.009454391896724701,
  0.1857212483882904,
  -0.20607724785804749,
  0.08724341541528702,
  0.2441616952419281,
  -0.3598603308200836,
  0.11515984684228897,
  -1.15869140625,
  -0.3733365833759308,
  -0.005698822438716888,
  -0.1037680134177208,
  -0.2733969986438751,
  0.22516557574272156,
  -0.7947500944137573,
  -0.03810849040746689,
  0.08164844661951065,
  -0.0687737688422203,
  0.44926688075065613,
  -0.05646636337041855,
  -0.6090433597564697,
  -0.9196134805679321,
  -0.05829193443059921,
  0.5445113182067871,
  0.13394126296043396,
  0.7902024984359741,
  -0.09965016692876816,
  -0.722817063331604,
  0.2825952470302582,
  -0.790690541267395,
  0.240751713514328,
  -0.6372673511505127,
  -0.313736230134964,
  0.2001769244670868,
  0.04571733623743057,
  0.339082807302475,
  0.2356146275997162,
  -0.04620959609746933,
  0.8224441409111023,
  0.20912107825279236,
  -0.1548046

In [6]:
import json

# === Extract scores and compute mean ===
bleurt_scores = results["scores"]
mean_bleurt = sum(bleurt_scores) / len(bleurt_scores)

# === Print and save results ===
print(f"BLEURT mean score: {mean_bleurt:.4f}")

output = {
    "bleurt_model": "bleurt-tiny-512",
    "mean_score": mean_bleurt,
    "scores": bleurt_scores,
    "num_samples": len(bleurt_scores),
}

with open("base_bleurt_tiny512_results.json", "w") as f:
    json.dump(output, f, indent=2)


BLEURT mean score: -0.0975
